In [1]:
import json
import datetime
import time
import requests
import nbimporter
import random
import Proxy_Gatherer as pg
import Init_database as indb
import Insert_py
import Get_py
import psycopg2

Importing Jupyter notebook from Proxy_Gatherer.ipynb
Importing Jupyter notebook from Init_database.ipynb
Importing Jupyter notebook from Insert_py.ipynb
Importing Jupyter notebook from Get_py.ipynb


In [2]:
# #--------THIS ONLY RUNS ONE TIME TO CREATE A DATABASE FRAMEWORK--------#
# import Create_Teld_Database as ctd
# db_init = ctd.create_teld_database()

In [3]:
#--------INITIALIZING CONNECTION TO DATABASE--------#
db_name="db_teld"
user_name="postgres"
password = "postgres"
host="127.0.0.1"
port="5432"
conn = indb.init_db(db_name, user_name, password, host, port)

In [4]:
##------- Specify request header ------- ##

hdr = {'User-Agent':'Mozilla/5.0 Chrome/\
59.0.3071.115 Safari/537.36', 
        'Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8'}
#------- Gathering proxies for visiting teld.cn -------##
gatherproxy = pg.ProxyGatherer(hdr)
proxies = gatherproxy.getproxies()
# t = time.time()
proxies_pool = gatherproxy.proxies_cleanup(proxies) # Could also specify timeout limit you prefer, 
                                                                #format--(connect time, read time)
# print(time.time()-t)  ## usually this checking process takes a minute or so and usable proxies are around 80+.
# proxies_pool=[{'http': '182.91.125.215:8080'}]

Got rid of 15 unusable proxies.


In [5]:
# print(len(proxies_pool))
# proxies_pool

In [6]:
## ----- WEB scraping demo -----##
# endTime = datetime.datetime.now() + datetime.timedelta(minutes=1)

# while True:
#     if datetime.datetime.now() >= endTime:
#         break
#     #--------GATHERING STATIONS DATA FROM APP--------#
#     station_info = Get_py.get_allStations(hdr)
#     #--------INSERTING STATIONS DATA--------#
#     conn = Insert_py.insert_station_info(conn, station_info)
#     #--------GATHERING PILE DATA FROM WEBSITE--------#
#     for station in station_info:
#         piles_info = Get_py.get_piles(station['code'], proxies_pool, hdr)
#         conn = Insert_py.insert_pile_info(conn, str(station['code']), piles_info)
#     print("time for a sleep zz..")
#     time.sleep(30)
    

In [7]:
# ##------- Every once a while runs it update station and pile fixed info -------##
# #--------------------------------------------#
# #------- Gather all stations  -------#
# station_info = Get_py.get_allStations(hdr)
# for station in station_info:
#     stationId = str(station['id'])  
#     print(stationId)
#     #------- Gather individual stations details -------#
#     station_detailedInfo = Get_py.get_oneStation_APP(hdr, stationId)
#     conn = Insert_py.insert_singStation_details(conn, station_detailedInfo)
#     #------- Gather individual pile code -------#
#     piles_info = Get_py.get_piles(stationId, proxies_pool, hdr)
#     conn = Insert_py.insert_pilecode(conn, stationId, piles_info)
# #------- Gather inidividual pile details -------#
# try:
#     cur = conn.cursor()
#     cur.execute("select pilecode from tbl_APP_pilecode;")
#     pilecode_list = cur.fetchall()
#     cur.close()
# except psycopg2.Error as e:
#     print(e)
    
# for pile in pilecode_list:
#     pile_detailedInfo = Get_py.get_onePile_APP(hdr, pile[0])
#     if pile_detailedInfo is not None:
#         conn = Insert_py.insert_singPile_details(conn, pile_detailedInfo)
#     else:
#         print("Running None Func.")
#         conn = Insert_py.insert_singPile_none(conn, pile_detailedInfo, pile[0])
# #51 Running None Func.

In [8]:
#------- Request pilecode from db for states -------#
try:
    cur = conn.cursor()
    cur.execute("select pilecode from tbl_APP_pilecode;")
    pilecode_list = cur.fetchall()
    cur.close()
except psycopg2.Error as e:
    print(e)

In [9]:
F = open('Error_log.txt','w') 

In [10]:
endTime = datetime.datetime.now() + datetime.timedelta(hours=6)
while True:
    if datetime.datetime.now() >= endTime:
        break
    for pile in pilecode_list:
        try:
            pile_detailedInfo, proxies_pool = Get_py.get_onePile_APP(F, hdr, pile[0], proxies_pool, time_out=1000)
        except ValueError as e:
            F.write(str(e))
            F.write('\n')
        if pile_detailedInfo is not None:
            pillCode = str(pile_detailedInfo['pillCode'])
            stateCode = str(pile_detailedInfo['stateCode'])
            stateCodeV3 = str(pile_detailedInfo['stateCodeV3'])
            if pile_detailedInfo['stateName'] is not None:
                stateName = str(pile_detailedInfo['stateName'].encode('utf-8'))
            else:
                stateName = ''
            if pile_detailedInfo['stateNameV3'] is not None:
                stateNameV3 = str(pile_detailedInfo['stateNameV3'].encode('utf-8'))
            else:
                stateNameV3 = ''
            conn = Insert_py.insert_state_info(conn, pillCode, stateCode, stateName, stateCodeV3, stateNameV3)
    print("time for a sleep zz..")
    time.sleep(600)
F.close()

failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed


ChunkedEncodingError: ("Connection broken: error(104, 'Connection reset by peer')", error(104, 'Connection reset by peer'))

In [ ]:
len(proxies_pool)
# F.close()

In [ ]:
# REQUEST_URL = 'http://baseapi.teld.cn/api/GetPillDetails/StaV2?pillCode='+pile[0]
# proxy_idx = random.randint(0,len(proxies_pool)-1)
# pile_resp = requests.get(REQUEST_URL, proxies=proxies_pool[proxy_idx], headers=hdr, timeout=1000)

In [ ]:
# pilecode = '1101060314110'
# pile_detailedInfo = Get_py.get_onePile_APP(hdr, pilecode)
# conn = Insert_py.insert_singPile_details(conn, pile_detailedInfo)
# REQUEST_URL = 'http://baseapi.teld.cn/api/GetPillDetails/StaV2?pillCode='+pilecode
# pile_resp = requests.get(REQUEST_URL, headers=hdr)


In [ ]:
# json_requestData = pile_resp.json()
# json_requestData
# pile_detailedInfo = json_requestData['data']['terminal']

In [ ]:
# stationId = '044a0e7c-419e-477e-aafb-8d4b9eea1475'
# piles_info = Get_py.get_piles(stationId, proxies_pool, hdr)
# pile_detailedInfo = Get_py.get_onePile_APP(hdr, piles_info[3]['code'])

In [ ]:
# pile_detailedInfo
# conn = Insert_py.insert_singPile_details(conn, pile_detailedInfo)

In [ ]:
##------- Try fetching all the data through psycopg2 -------## 
# try:
#     cur = conn.cursor()
#     cur.execute("select stationid from tbl_app_eachstations limit 10;")
#     exists = cur.fetchall()
#     cur.close()
# except psycopg2.Error as e:
#     print(e)
# for i in range(len(exists)):
#     print(exists[i][0])

In [ ]:
# REQUEST_URL = 'https://www.teld.cn/StationNetwork/GetStationNetword?ProvinceName=&CityName=&KeyWords=&RegionName=& \
#                             type=&page=1&rows=10'
# # req = urllib.request.Request(REQUEST_URL) 
# hdr = {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/ \
#                        59.0.3071.115 Safari/537.36', 
#        'Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8'}
# response = urllib2.Request(REQUEST_URL, headers=hdr)

# try:
#     page = urllib2.urlopen(response)
# except urllib2.HTTPError, e:
#     print e.fp.read()
    
# content = json.load(page)
# print(content['rows'][0])
# response_data = urllib.request.urlopen(REQUEST_URL)
# d = json.load(response_data)
# print('number of results %d' % (len(d)))
# type(response_data)